<a href="https://colab.research.google.com/github/HarshiniAiyyer/WinnieDaPooh/blob/main/2_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2: Feature Engineering

## Reading and Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/HarshiniAiyyer/WinnieDaPooh/refs/heads/main/hdata.csv')
df.head()

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,SrcPkts,Location
0,2021-05-18 02:00:02.267954,0.000038,tcp,116.196.102.218,659,->,159.65.26.180,6379,S_RA,0,0,2,128,74,1,London
1,2021-05-18 02:00:12.072882,0.027066,tcp,89.248.165.204,44109,->,159.65.26.180,6450,SR_RA,0,0,3,166,112,2,London
2,2021-05-18 02:00:15.309728,0.052794,tcp,85.119.151.250,56013,->,159.65.26.180,8080,SR_RA,0,0,3,166,112,2,London
3,2021-05-18 02:00:16.039044,0.073921,tcp,85.119.151.250,56013,->,159.65.26.180,2169,SR_RA,0,0,3,166,112,2,London
4,2021-05-18 02:00:16.543067,0.074616,tcp,85.119.151.250,56013,->,159.65.26.180,3128,SR_RA,0,0,3,166,112,2,London


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26040 entries, 0 to 26039
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   StartTime  26040 non-null  object 
 1   Dur        26040 non-null  float64
 2   Proto      26040 non-null  object 
 3   SrcAddr    26040 non-null  object 
 4   Sport      26040 non-null  int64  
 5   Dir        26040 non-null  object 
 6   DstAddr    26040 non-null  object 
 7   Dport      26040 non-null  int64  
 8   State      26040 non-null  object 
 9   sTos       26040 non-null  int64  
 10  dTos       26040 non-null  int64  
 11  TotPkts    26040 non-null  int64  
 12  TotBytes   26040 non-null  int64  
 13  SrcBytes   26040 non-null  int64  
 14  SrcPkts    26040 non-null  int64  
 15  Location   26040 non-null  object 
dtypes: float64(1), int64(8), object(7)
memory usage: 3.2+ MB


## Extracting Temporal Features

### Hour and day of the week

In [5]:
df['StartTime'] = pd.to_datetime(df['StartTime'])

In [6]:
df['Day'] = df['StartTime'].dt.day_name()
df['Weekend?'] = df['Day'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

In [7]:
df.head()

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,SrcPkts,Location,Day,Weekend?
0,2021-05-18 02:00:02.267954,0.000038,tcp,116.196.102.218,659,->,159.65.26.180,6379,S_RA,0,0,2,128,74,1,London,Tuesday,0
1,2021-05-18 02:00:12.072882,0.027066,tcp,89.248.165.204,44109,->,159.65.26.180,6450,SR_RA,0,0,3,166,112,2,London,Tuesday,0
2,2021-05-18 02:00:15.309728,0.052794,tcp,85.119.151.250,56013,->,159.65.26.180,8080,SR_RA,0,0,3,166,112,2,London,Tuesday,0
3,2021-05-18 02:00:16.039044,0.073921,tcp,85.119.151.250,56013,->,159.65.26.180,2169,SR_RA,0,0,3,166,112,2,London,Tuesday,0
4,2021-05-18 02:00:16.543067,0.074616,tcp,85.119.151.250,56013,->,159.65.26.180,3128,SR_RA,0,0,3,166,112,2,London,Tuesday,0


### Deriving Session Frequency per hour

Rate at which a source address initiates new sessions / convos per  per given time window. Here we wud take an hourly bucket (here).

Why?
- Shorter size of continuous session frequency from the same IP may indicate a frauduent activity or a bot activity.

In [8]:
df['HourBin'] = df['StartTime'].dt.floor('h')

So it is Source Address initiated per hour. Size of this group is the frequency of sessions initiated by the source address per hour.

Using Size, not count because we want to count rows with null values as well.

In [9]:
sessfreq = df.groupby(['SrcAddr', 'HourBin']).size().reset_index(name = 'SessionFrequency')

In [10]:
df = df.merge(sessfreq, on=['SrcAddr', 'HourBin'], how='left')

### Inter-Arrival Time Between Flows

It measures the time difference between one connection and the next one from the same IP.

#### ✅ Why it’s useful:
Regular gaps may indicate bot activity or beaconing (malware phoning home).

Very small gaps suggest port scanning or flooding.

In [11]:
df = df.sort_values(by=['SrcAddr', 'StartTime']).reset_index(drop=True)

# Time difference per start time per source address
df['InterArrivalTime'] = df.groupby('SrcAddr')['StartTime'].diff().dt.total_seconds()

# fill null values with 0
df['InterArrivalTime'] = df['InterArrivalTime'].fillna(0)

## Flow Features

### Bytes Per Packet

In [12]:
df['BytesPerPacket'] = df['TotBytes'] / df['TotPkts']

### Source Byte Ratio

In [13]:
df['SrcByteRatio'] = df['SrcBytes'] / df['TotBytes']

## Duration Profiling

- We will venture a little bit into duration and flows.
- Short bursts of flows may imply suspicious signals

#### Set thresholds for durations

In [14]:
df['FlowType'] = pd.cut(df['Dur'],
                        bins=[0, 0.05, 1, 10, df['Dur'].max()],
                        labels=['Very Short', 'Short', 'Medium', 'Long'])


In [15]:
df['FlowType'].value_counts()

,count
FlowType,
Very Short,17076
Short,8406
Medium,420
Long,137


In [16]:
short = df[df['FlowType'] == 'Short']

Potential EDA on this:

- Number of distinct destination ports/IPs touched  ```short['DstPort'].nunique()```


- How many such short flows per source IP ```short['SrcIP'].value_counts()```


- Time gap between bursts (inter-arrival times) ```short['InterArrivalTime'].describe()```

Data for Anomaly Detection:

Use short-duration thresholds to flag IPs with many tiny-duration flows, especially if:

- They're hitting many destination ports

- The flows are regularly spaced (possible bot/script)

- They're from suspicious ASNs or locations

This duration-based profiling can feed directly into your unsupervised model (e.g., Isolation Forest), or even be visualized to support anomaly triage.

## Network Behavior

### Unique destination ports accessed per source IP

In [17]:
uniq = df.groupby(['SrcAddr'])['Dport'].nunique()
uniq.describe()

,Dport
count,5998.000000
mean,3.067356
std,22.037815
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1375.000000


### Count of unique destination IPs per source IP

In [18]:
uniq1 = df.groupby(['SrcAddr'])['DstAddr'].nunique()
uniq1.describe()


,DstAddr
count,5998.000000
mean,1.304935
std,0.638030
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,3.000000


### Unique Destination Ports per Source IP

In [19]:
uniq2 = df.groupby('SrcAddr')['Dport'].nunique().reset_index(name='UniqueDstPorts')
uniq2

,SrcAddr,UniqueDstPorts
0,1.0.202.88,1
1,1.0.218.64,1
2,1.10.184.106,1
3,1.116.14.134,1
4,1.116.140.153,1
...,...,...
5993,96.52.118.130,1
5994,96.59.111.169,1
5995,96.91.105.238,1
5996,98.113.35.10,1


### Half Open Connections

In TCP, a three-way handshake establishes a connection:

- SYN
- SYN-ACK
- ACK


A half-open connection occurs when:

- The initiator sends a SYN
- The responder replies with SYN-ACK
- But the final ACK is not received

➡️ Often used in stealth scans (e.g., SYN scans).

#### Why do we need to find out half-open connections?

- Ideally, SYN scans are done to detect half open ports, these ports are then used to launch attacks.

- Because they don't complete the full SYN- ACK handshake, they might evade detection from some firewalls or intrusion detection systems (IDS).

| State Code | Interpretation                       | Half-Open? | Notes                                                           |
| ---------- | ------------------------------------ | ---------- | --------------------------------------------------------------- |
| `S_RA`     | SYN sent, RST+ACK received           | ✅ Yes      | Target **refused** connection – could be result of **SYN scan** |
| `SR_RA`    | SYN-ACK sent, RST+ACK received       | ✅ Yes      | Target tried to complete connection, but initiator **reset** it |
| `SA_R`     | SYN-ACK received, then RST           | ✅ Yes      | Another common **SYN scan** pattern                             |
| `SR_`      | SYN-ACK sent, no further response    | ✅ Yes      | Strong indicator of **unacknowledged SYN** — **half-open**      |
| `SRA_RA`   | SYN, RST, ACK received; RST+ACK sent | ✅ Yes      | Messy teardown after handshake; likely a scanner                |


In [20]:
df['HalfOpen'] = df['State'].isin(['S_RA', 'SR_RA', 'SA_R', 'SR_', 'SRA_RA'])


### Trying to spot scanners

In [21]:
df.groupby('SrcAddr')['HalfOpen'].sum().sort_values(ascending=False)

,HalfOpen
SrcAddr,
45.143.200.34,1376
201.149.21.179,419
62.210.189.127,393
45.155.205.79,363
195.54.161.150,322
...,...
77.247.110.174,0
80.193.40.189,0
81.97.59.20,0
